In [1]:
# Cell 1: Install Required Libraries
!pip install -q scikit-learn matplotlib torch transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.0 MB/s eta 0:00:00


In [11]:
#https://huggingface.co/Cohere/Cohere-embed-english-v3.0
!pip install -U cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.5 MB/s eta 0:00:00


In [12]:
# Cell 2: Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re
import cohere
import pandas as pd
from scipy.stats import spearmanr
import math

In [13]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Cell 3: Define Utility Functions
def simple_clean(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
from google.colab import userdata
cohere_key =  userdata.get('COHERE_API_KEY')  #Get your API key from www.cohere.com
co = cohere.Client(cohere_key)

In [39]:
# Cell 4: Load Data
df = pd.read_excel('/content/synthetic_resume_summaries (1).xlsx').iloc[:70]
all_texts = df["text"].tolist()
all_summaries = df["summary"].tolist()
print(f"Number of samples: {len(all_texts)}")

Number of samples: 70


In [40]:
# Cell 5: Split Data into Train and Test
num_samples = len(all_texts)
test_size = 30
if num_samples > test_size:
    train_indices, test_indices = train_test_split(range(num_samples), test_size=test_size, random_state=42)
else:
    test_indices = list(range(num_samples))
    print(f"Warning: Number of samples {num_samples} is less than or equal to test_size {test_size}, using all as test.")
print(f"Train indices: {len(train_indices)}")
print(f"Test indices: {len(test_indices)}")

Train indices: 40
Test indices: 30


In [41]:
# Cell 6: Prepare Corpus and Queries
corpus = all_texts
queries = [all_summaries[i] for i in test_indices]
print(f"Corpus size: {len(corpus)}")
print(f"Query size: {len(queries)}")

Corpus size: 70
Query size: 30


In [42]:
# Cell 7: Load Model
model_name = "embed-english-v3.0"
print(f"Model loaded: {model_name}")

Model loaded: embed-english-v3.0


In [43]:
# Encode Corpus and Queries using Cohere
corpus_embeddings = co.embed(texts=[simple_clean(doc) for doc in corpus], input_type="search_document", model="embed-english-v3.0").embeddings
corpus_embeddings_np = np.asarray(corpus_embeddings)
print(f"Corpus embeddings shape: {corpus_embeddings_np.shape}")

Corpus embeddings shape: (70, 1024)


In [44]:
query_embeddings = co.embed(texts=[simple_clean(q) for q in queries], input_type="search_query", model="embed-english-v3.0").embeddings
query_embeddings_np = np.asarray(query_embeddings)
print(f"Query embeddings shape: {query_embeddings_np.shape}")

Query embeddings shape: (30, 1024)


In [45]:
# Compute Cosine Similarities
similarities = cosine_similarity(query_embeddings_np, corpus_embeddings_np)
print(f"Similarities matrix shape: {similarities.shape}")

Similarities matrix shape: (30, 70)


In [46]:
# Cell 10: Calculate Performance Metrics
num_queries = len(queries)
mrr_total = 0.0
ap_total = 0.0
ndcg_total = 0.0
top1_correct = 0
spearman_total = 0.0

for k in range(num_queries):
    ranked_indices = np.argsort(similarities[k, :])[::-1]
    # MRR
    rank = np.where(ranked_indices == test_indices[k])[0]
    if len(rank) > 0:
        rank = rank[0] + 1
        mrr_total += 1.0 / rank
    # MAP
    sum_prec = 0.0
    rel_found = 0
    for position, doc_id in enumerate(ranked_indices):
        if doc_id == test_indices[k]:
            rel_found += 1
            sum_prec += rel_found / (position + 1)
    if rel_found > 0:
        ap = sum_prec / rel_found
    else:
        ap = 0.0
    ap_total += ap
    # NDCG
    rel = [1 if doc_id == test_indices[k] else 0 for doc_id in ranked_indices]
    dcg = sum([rel[i] / math.log2(i + 2) for i in range(len(rel))])
    idcg = 1.0 / math.log2(2)
    ndcg = dcg / idcg if idcg > 0 else 0.0
    ndcg_total += ndcg
    # Top-1 Accuracy
    if ranked_indices[0] == test_indices[k]:
        top1_correct += 1
    # Spearman
    scores = similarities[k, :]
    true_labels = [1 if j == test_indices[k] else 0 for j in range(len(corpus))]
    rho, p = spearmanr(scores, true_labels)
    if not math.isnan(rho):
        spearman_total += rho

mrr = mrr_total / num_queries
map_score = ap_total / num_queries
average_ndcg = ndcg_total / num_queries
accuracy = top1_correct / num_queries
average_spearman = spearman_total / num_queries

In [47]:
print("Performance Summary:")
print(f"- MRR: {mrr:.4f}")
print(f"- MAP: {map_score:.4f}")
print(f"- NDCG: {average_ndcg:.4f}")
print(f"- Top-1 Accuracy: {accuracy:.4f}")
print(f"- Average Spearman: {average_spearman:.4f}")

Performance Summary:
- MRR: 1.0000
- MAP: 1.0000
- NDCG: 1.0000
- Top-1 Accuracy: 1.0000
- Average Spearman: 0.2056
